In [501]:
"""
From: Boyuan Lyu
To: Dashmote
Title: Tripadvisor case
"""

'\nFrom: Boyuan Lyu\nTo: Dashmote\nTitle: Tripadvisor case\n'

In [502]:
from urllib.request import urlopen
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
import html
import base64

# 1. Get url of all bars on all pages

In [503]:
def get_all_pages(input_page1_url):
    """
    enter 'Shanghai' in search bar and tick Bars & Pubs to get page1 url
    function returns urls of all pages
    """  
    page1_request = requests.get(input_page1_url)
    page1_content = page1_request.content
    page1_Soup = BeautifulSoup(page1_content, "lxml")

    output_all_pages_url = [] #page 2 and following
    page_url = page1_Soup.findAll(class_="pageNum taLnk")
    for i in page_url:
        output_all_pages_url.append(i.get('href'))
    output_all_pages_url = ['https://www.tripadvisor.com' + i for i in output_all_pages_url]
    output_all_pages_url = [input_page1_url] + output_all_pages_url
    
    return output_all_pages_url

page1_url = r'https://www.tripadvisor.com/Restaurants-g308272-zfg11776-Shanghai.html' #page 1
all_pages_url = get_all_pages(page1_url)
all_pages_url

['https://www.tripadvisor.com/Restaurants-g308272-zfg11776-Shanghai.html',
 'https://www.tripadvisor.com/Restaurants-g308272-oa30-zfg11776-Shanghai.html#EATERY_LIST_CONTENTS',
 'https://www.tripadvisor.com/Restaurants-g308272-oa60-zfg11776-Shanghai.html#EATERY_LIST_CONTENTS',
 'https://www.tripadvisor.com/Restaurants-g308272-oa90-zfg11776-Shanghai.html#EATERY_LIST_CONTENTS',
 'https://www.tripadvisor.com/Restaurants-g308272-oa120-zfg11776-Shanghai.html#EATERY_LIST_CONTENTS']

In [504]:
def get_all_bars_url(input_all_pages_url):
    """
    function returns url of each bar
    """  
    output_all_bars_url = []
    
    for i in input_all_pages_url:
        page_request = requests.get(i)
        page_content = page_request.content
        page_Soup = BeautifulSoup(page_content, "lxml")
        bar_url = page_Soup.findAll(class_="_15_ydu6b")
        for i in bar_url:
            output_all_bars_url.append(i.get('href'))
    output_all_bars_url = ['https://www.tripadvisor.com' + i for i in output_all_bars_url]
    
    return output_all_bars_url

all_bars_url = get_all_bars_url(all_pages_url)
all_bars_url

['https://www.tripadvisor.com/Restaurant_Review-g308272-d11676778-Reviews-The_Ritz_Bar_Lounge-Shanghai.html',
 'https://www.tripadvisor.com/Restaurant_Review-g308272-d2544147-Reviews-Blue_Frog_Daning-Shanghai.html',
 'https://www.tripadvisor.com/Restaurant_Review-g308272-d8444760-Reviews-Zapfler_Craft_Beer_Brewery-Shanghai.html',
 'https://www.tripadvisor.com/Restaurant_Review-g308272-d4713210-Reviews-Kartel_Wine_Bar-Shanghai.html',
 'https://www.tripadvisor.com/Restaurant_Review-g308272-d8133799-Reviews-CAGES_Bar_and_Sports-Shanghai.html',
 'https://www.tripadvisor.com/Restaurant_Review-g308272-d1828779-Reviews-Zapata_s_Shanghai-Shanghai.html',
 'https://www.tripadvisor.com/Restaurant_Review-g308272-d15239610-Reviews-Stone_Brewing_Tap_Room-Shanghai.html',
 'https://www.tripadvisor.com/Restaurant_Review-g308272-d3465833-Reviews-XiErDun_XingZheng_JiuLang-Shanghai.html',
 'https://www.tripadvisor.com/Restaurant_Review-g308272-d7224423-Reviews-Yu_Bar_Renaissance_Shanghai_Yu_Garden_Hotel-S

# 2. Get all info of each bar

In [505]:
def get_bar_info(input_bar_url):
    """
    function returns all desired info of each bar
    """  
    request = requests.get(input_bar_url)
    content = request.content
    Soup = BeautifulSoup(content, "lxml")
    
    #bar name
    try:
        name_text = Soup.findAll('h1')
        name_clean = re.sub('<[^>]+>', '', str(name_text[1]))
        name_unescape = html.unescape(name_clean)
    except IndexError:
        name = 'NaN'
    else:
        name = name_clean

    #area
    try:
        area_text = Soup.findAll('span', {'class': '_2saB_OSe _1OBMr94N'})
        area_clean = re.sub('<[^>]+>', '', str(area_text))
        area_unescape = html.unescape(area_clean).strip('[]')
    except IndexError:
        area_unescape = 'NaN'
    else:
        area_unescape = area_unescape

    #address
    try:
        address_text = Soup.findAll('a', {'href': '#MAPVIEW'})
        address_clean = re.sub('<[^>]+>', '', str(address_text))
        address_unescape = html.unescape(address_clean)
        address = address_unescape.strip('[]')
    except IndexError:
        address = 'NaN'
    else:
        address = address

    #rating
    try:
        rating_text = Soup.findAll('span', {'class': 'r2Cf69qf'})
        rating_clean = re.sub('<[^>]+>', '', str(rating_text))
        rating_unescape = html.unescape(rating_clean)
        rating = rating_unescape.strip('[]')[:3]
    except IndexError:
        rating = 'NaN'
    else:
        if rating == '':
            rating = 'NaN'
        else:
            rating = rating

    #cuisine
    try:
        cuisine_text = Soup.find('div',text='CUISINES')
        cuisine_clean = re.sub('<[^>]+>', '', str(cuisine_text.find_next_sibling("div")))
        cuisine_unescape = html.unescape(cuisine_clean)
    except AttributeError:
        cuisine_unescape = 'NaN'
    else:
        cuisine_unescape = cuisine_unescape
    
    #review
    try:
        review_text = Soup.findAll('a', {'href': '#REVIEWS'})
        review_clean = re.sub('<[^>]+>', '', str(review_text[-1]))
        review_unescape = html.unescape(review_clean)
        review = review_unescape.strip('[reviews] ' or '[review] ')
    except IndexError:
        review = 'NaN'
    else:
        review = review

    #telephone
    try:
        tel_text = Soup.findAll('a', href = re.compile("tel:"))
        tel_clean = re.sub('<[^>]+>', '', str(tel_text[0]))
        tel_unescape = html.unescape(tel_clean)
    except IndexError:
        tel_unescape = 'NaN'
    else:
        tel_unescape = tel_unescape

    #website
    try:
        web_text = Soup.findAll('a')
        web_list = [i.get('data-encoded-url') for i in web_text]
        web_list_clean = [i for i in web_list if i != None]
        encoded_url = web_list_clean[0]
        decoded_url = base64.b64decode(encoded_url).decode('UTF-8')
        start_index = decoded_url.index('http')
        web_url = decoded_url[start_index:-5]
    except IndexError:
        web_url = 'NaN'
    else:
        if 'maps.google' in web_url:
            web_url = 'NaN'
        else:
            web_url = web_url

    output_bar_info =  {
            'Bar Name': name, 
            'Url': input_bar_url, 
            'Area': area_unescape,
            'Address': address, 
            'Cuisine': cuisine_unescape, 
            'Number of Reviews': review, 
            'Rating': rating, 
            'Telephone': tel_unescape, 
            'Website': web_url
    }
    return output_bar_info

# 3. Save and finish

In [506]:
"""
save all info and done
"""
all_bars_info = []
for i in all_bars_url:
    bar_info = get_bar_info(i)
    all_bars_info.append(bar_info)
df = pd.DataFrame.from_dict(all_bars_info) 
df.to_csv(r'C:\Users\cnsghlubo\Desktop\tripadvisor_shanghai_bar.csv', header=True, index=True) 